In [1]:
pip install lxml

     |████████████████████████████████| 5.8MB 8.6MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy

     |████████████████████████████████| 112kB 5.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [152]:
import requests
import pandas as pd
pd.options.display.max_rows= None
pd.options.display.max_columns= None
import lxml
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Reading in the first table from URL

In [3]:
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results= requests.get(url)
df= pd.read_html(results.text)

- How many tables does our list contain?

In [4]:
len(df)

3

- Clearly, we need to deal with only the following table:

In [5]:
df[0]

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [6]:
df= df[0]
print('Shape of our dataframe is', df.shape)
print('Number of NA values present in our df is', df.isna().sum().sum())

Shape of our dataframe is (180, 3)
Number of NA values present in our df is 77


## Carrying out step #3 in assignment:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [7]:
df.columns

Index(['Postal code', 'Borough', 'Neighborhood'], dtype='object')

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [8]:
df= df[df['Borough']!='Not assigned']

In [9]:
print('Shape of our dataframe is', df.shape)
print('Number of NA values present in our df is', df.isna().sum().sum())

Shape of our dataframe is (103, 3)
Number of NA values present in our df is 0


- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    - Our transformed dataframe does not contain any NA values in nay columns.
    - However it might contain 'Not assigned' value in 'Neighborhood' column, we should check for that.

In [10]:
'Not assigned' in df['Neighborhood'].unique().tolist()

False

- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
    - Seems wikipedia page has been updated to reflect what was required in the assignment.
    - We are simply replacing the '/' by a ','

In [11]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [12]:
df['Neighborhood']= df['Neighborhood'].str.replace('/', ',')
df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    - No assumptions were made, markdown cells were added, notebook cleaned.

- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [13]:
print('Number of rows in our dataframe:', df.shape[0])

Number of rows in our dataframe: 103


## =============================== PART 1 complete ===============================

## Getting the latitude and the longitude coordinates of each neighborhood:

- Since the Geocoder package is said to be unreliable, we shall be using the csv file @ https://cocl.us/Geospatial_data.

- Use the Geocoder package or the csv file to create the following dataframe:

In [14]:
lldf= pd.read_csv('https://cocl.us/Geospatial_data')
lldf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
lldf.shape

(103, 3)

- Both our dataframes contain equal number of rows. We plan on joining both on the 'Postal Code' column.
- We cannot be sure whether that column is identical or not in both the dataframes.
    - We can convert that column from both datarames into sets & check for equality.

In [16]:
set(df['Postal code']) == set(lldf['Postal Code'])

True

- Now we can proceed with our join:

In [17]:
df= pd.merge(df, lldf, left_on= ['Postal code'], right_on= ['Postal Code']).drop(['Postal Code'], axis= 1)

In [18]:
df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## ================================= PART 2 complete =================================

- Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

In [19]:
print('Total number of boroughs:', len(df['Borough'].unique()))
print('Total number of neighborhoods:', len(df['Neighborhood'].unique()))

Total number of boroughs: 10
Total number of neighborhoods: 98


In [20]:
print('Boroughs in our dataframe:', df['Borough'].unique())

Boroughs in our dataframe: ['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Mississauga']


- Create a new df where the 'Borough' column contains values having the word 'Toronto' in it:

In [21]:
tdf= pd.DataFrame(columns= df.columns)

for i, r in df.iterrows():
    if 'Toronto' in r['Borough']:
        tdf.loc[len(tdf)]= r

In [22]:
tdf['Borough'].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

In [23]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
loc = geolocator.geocode(address)
lat = loc.latitude
lon = loc.longitude
print(f'The geograpical coordinate of Toronto are {lat}, {lon}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [24]:
tdf.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


- Visualizing the neighborhoods having Borough names 'Toronto' in them:

In [26]:
# create map of Manhattan using latitude and longitude values
map_tor = folium.Map(location=[lat, lon], zoom_start= 11)

# add markers to map
for lat, lng, label in zip(tdf['Latitude'], tdf['Longitude'], tdf['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

In [27]:
cid = 'V5CPX1X5D42YLFF3TQHEDVMPKEUQG5DGY00VQVSTO5BOOR4O' # your Foursquare ID
sec = 'R4CDEAYXLZ4MQPTK0SHOSZBR1YCXOUS5TYHNUWVLLYMZQBBP' # your Foursquare Secret
v = '20200101' # Foursquare API version

- Exploring the neighborhoods of the first neighborhood:

In [31]:
n_name= tdf.loc[0, 'Neighborhood']
n_lon= tdf.loc[0, 'Longitude']
n_lat= tdf.loc[0, 'Latitude']

url = f'https://api.foursquare.com/v2/venues/explore?client_id={cid}&client_secret={sec}&ll={n_lat},{n_lon}&v={v}&radius=500&limit=100'
url

'https://api.foursquare.com/v2/venues/explore?client_id=V5CPX1X5D42YLFF3TQHEDVMPKEUQG5DGY00VQVSTO5BOOR4O&client_secret=R4CDEAYXLZ4MQPTK0SHOSZBR1YCXOUS5TYHNUWVLLYMZQBBP&ll=43.6542599,-79.3606359&v=20200101&radius=500&limit=100'

In [36]:
results = requests.get(url).json()

In [47]:
def get_cat(row):
    print(ndf[0][0]['name'])

In [60]:
ndf= pd.json_normalize(results['response']['groups'][0]['items']) #flattening the json
headers=['venue.name', 'venue.location.lat', 'venue.location.lng', 'venue.categories']
ndf= ndf[headers]
for i, r in ndf.iterrows():
    ndf.loc[i, 'venue.categories']= r['venue.categories'][0]['name']
ndf.head()

,venue.name,venue.location.lat,venue.location.lng,venue.categories
0,Roselle Desserts,43.653447,-79.362017,Bakery
1,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


- Now we shall be performing the above for all the neighborhoods in tdf dataframe.
- We rae storing the new information into a new dataframe called venues.

In [72]:
venues= pd.DataFrame()

In [73]:
for i, r in tdf.iterrows():
    
    lat, lon= r['Latitude'], r['Longitude']
    url = f'https://api.foursquare.com/v2/venues/explore?client_id={cid}&client_secret={sec}&ll={lat},{lon}&v={v}&radius=500&limit=100'
    results = requests.get(url).json()
    
    ndf= pd.json_normalize(results['response']['groups'][0]['items']) #flattening the json
    headers=['venue.name', 'venue.location.lat', 'venue.location.lng', 'venue.categories']
    ndf= ndf[headers]
    ndf['Neighborhood']= r['Neighborhood']
    ndf['Neighborhood.lat']= lat
    ndf['Neighborhood.lon']= lon
    for i, r in ndf.iterrows():
        ndf.loc[i, 'venue.categories']= r['venue.categories'][0]['name']
    #ndf['Neighborhood']= r['Neighborhood']
    #ndf['Neighborhood.lat']= lat
    #ndf['Neighborhood.lon']= lon
    
    venues= pd.concat([venues, ndf], ignore_index= True)

In [75]:
venues.head()

,venue.name,venue.location.lat,venue.location.lng,venue.categories,Neighborhood,Neighborhood.lat,Neighborhood.lon
0,Roselle Desserts,43.653447,-79.362017,Bakery,"Regent Park , Harbourfront",43.65426,-79.360636
1,Tandem Coffee,43.653559,-79.361809,Coffee Shop,"Regent Park , Harbourfront",43.65426,-79.360636
2,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,"Regent Park , Harbourfront",43.65426,-79.360636
3,Body Blitz Spa East,43.654735,-79.359874,Spa,"Regent Park , Harbourfront",43.65426,-79.360636
4,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot,"Regent Park , Harbourfront",43.65426,-79.360636


In [76]:
# reordering the columns
venues = venues.reindex(columns= ['Neighborhood',	'Neighborhood.lat',	'Neighborhood.lon', 'venue.name', 'venue.location.lat',	'venue.location.lng', 'venue.categories'])

print('Dimensions of the venue dataframe:', venues.shape)

Dimensions of the venue dataframe: (1612, 7)


- Cmhecking if we have covered all the neighborhoods from tdf dataframe in venues dataframe successfully:

In [77]:
set(tdf['Neighborhood'])==set(venues['Neighborhood'])

True

- Let's check how many venues were returned for each neighborhood

In [78]:
venues.groupby(['Neighborhood']).agg({'venue.name': 'count'})

,venue.name
Neighborhood,
Berczy Park,58
"Brockton , Parkdale Village , Exhibition Place",22
Business reply mail Processing CentrE,17
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",16
Central Bay Street,67
Christie,17
Church and Wellesley,71
"Commerce Court , Victoria Hotel",100
Davisville,36


- Let's find out how many unique categories can be curated from all the returned venues

In [79]:
print(len(venues['venue.categories'].unique()))
venues['venue.categories'].unique()

230


array(['Bakery', 'Coffee Shop', 'Distribution Center', 'Spa',
       'Breakfast Spot', 'Restaurant', 'Park', 'Historic Site', 'Pub',
       'Farmers Market', 'Chocolate Shop', 'Dessert Shop', 'Theater',
       'Performing Arts Venue', 'Gym / Fitness Center',
       'French Restaurant', 'Café', 'Mexican Restaurant', 'Event Space',
       'Yoga Studio', 'Ice Cream Shop', 'Asian Restaurant', 'Shoe Store',
       'Art Gallery', 'Cosmetics Shop', 'Electronics Store', 'Bank',
       'Beer Store', 'Hotel', 'Health Food Store', 'Antique Shop',
       'Italian Restaurant', 'Beer Bar', 'Sushi Restaurant', 'Creperie',
       'Arts & Crafts Store', 'Burrito Place', 'Diner', 'Hobby Shop',
       'Discount Store', 'Fried Chicken Joint', 'Burger Joint',
       'Juice Bar', 'Sandwich Place', 'Gym', 'College Auditorium', 'Bar',
       'College Cafeteria', 'Clothing Store', 'Comic Shop', 'Plaza',
       'Tea Room', 'Music Venue', 'Ramen Restaurant', 'Thai Restaurant',
       'Steakhouse', 'Sporting Good

In [80]:
# one hot encoding
tor_oh= pd.get_dummies(venues['venue.categories'], prefix= '', prefix_sep= '')

# add neighborhood column back to dataframe
tor_oh['Neighborhood.name']= venues['Neighborhood'] 

cols= ['Neighborhood.name'] + tor_oh.drop(['Neighborhood.name'], axis= 1).columns.tolist()
tor_oh= tor_oh[cols]

tor_oh.head()

,Neighborhood.name,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

- And let's examine the new dataframe size.

In [81]:
tor_oh.shape

(1612, 231)

- Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [82]:
tor_grouped = tor_oh.groupby('Neighborhood.name').mean().reset_index()
tor_grouped

,Neighborhood.name,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.017241,0.034483,0.000000,0.000000,0.00,0.017241,0.017241,0.000000,0.034483,0.000000,0.000000,0.000000,0.017241,0.0000,0.000000,0.00000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.017241,0.051724,0.068966,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.017241,0.000000,0.017241,0.000000,0.000000,0.000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241

- Let's confirm the new size

In [83]:
tor_grouped.shape

(39, 231)

- Let's print each neighborhood along with the top 5 most common venues:

In [85]:
num_top = 5

for hood in tor_grouped['Neighborhood.name']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood.name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.03
3              Bakery  0.03
4      Farmers Market  0.03


----Brockton , Parkdale Village , Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Intersection  0.05
4         Stadium  0.05


----Business reply mail Processing CentrE----
                venue  freq
0         Yoga Studio  0.06
1                 Spa  0.06
2       Garden Center  0.06
3              Garden  0.06
4  Light Rail Station  0.06


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4     Boat or Ferry  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.19
1  Italian Restaurant  0.06
2  

- Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [127]:
# creating the new dataframe tor_venues with 10 most popular venues
n_top= 10

headers= ['Neighborhood']
for i in range(n_top):
    headers.append(f'Popular venue #{i+1}')

In [129]:
tor_venues= pd.DataFrame(columns= headers)

for hood in tor_grouped['Neighborhood.name']:

    temp= tor_grouped[tor_grouped['Neighborhood.name']==hood].T.reset_index()
    temp= temp.iloc[1:]
    temp.columns = ['venue','freq']
    row= []
    row.append(hood)
    row= row+temp.sort_values(['freq'], ascending= False).head(10).loc[:, 'venue'].T.tolist()
    
    tor_venues= tor_venues.append(pd.Series(row, index= tor_venues.columns), ignore_index= True)    
    
tor_venues.head()

,Neighborhood,Popular venue #1,Popular venue #2,Popular venue #3,Popular venue #4,Popular venue #5,Popular venue #6,Popular venue #7,Popular venue #8,Popular venue #9,Popular venue #10
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Farmers Market,Italian Restaurant,Beer Bar,Restaurant,Café,Cheese Shop
1,"Brockton , Parkdale Village , Exhibition Place",Café,Breakfast Spot,Coffee Shop,Intersection,Stadium,Furniture / Home Store,Climbing Gym,Nightclub,Burrito Place,Restaurant
2,Business reply mail Processing CentrE,Yoga Studio,Spa,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Park,Pizza Place,Restaurant
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Airport,Boat or Ferry,Sculpture Garden,Rental Car Location,Plane,Airport Food Court,Coffee Shop
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Department Store,Thai Restaurant,Fried Chicken Joint,Bubble Tea Shop,Japanese Restaurant


- Now we cluster the neighborhoods using K-Means:

In [134]:
tor_grouped.head()

,Neighborhood.name,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,0.017241,0.000000,0.0,0.0,0.000000,0.017241,0.0,0.017241,0.034483,0.0,0.000000,0.0,0.017241,0.017241,0.0,0.034483,0.0,0.0,0.000000,0.017241,0.0000,0.0,0.0000,0.0,0.017241,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.034483,0.0,0.0,0.0,0.034483,0.0,0.0,0.0,0.000000,0.017241,0.051724,0.068966,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.000000,0.017241,0.000000,0.0,0.0,0.017241,0.0,0.0,0.0,0.000000,0.000000,0.0,0.017241,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017241,0.0,0.0,0.0,0.000000,0.034483,0.000000,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.017241,0.017241,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0

In [136]:
# set number of clusters
k = 5

tor_clustered = tor_grouped.drop(['Neighborhood.name'], axis= 1)
kmeans = KMeans(n_clusters= k, random_state= 0).fit(tor_clustered)

# check cluster labels generated for the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0,
       0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0], dtype=int32)

- Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [146]:
# tor_venues is the left dataframe to maintain kmeans label orders
new_df= pd.merge(tor_venues, tdf, left_on= 'Neighborhood', right_on= 'Neighborhood').drop(['Postal code'], axis= 1)
new_df['Cluster_label']= kmeans.labels_

In [148]:
new_df

,Neighborhood,Popular venue #1,Popular venue #2,Popular venue #3,Popular venue #4,Popular venue #5,Popular venue #6,Popular venue #7,Popular venue #8,Popular venue #9,Popular venue #10,Borough,Latitude,Longitude,Cluster_label
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Farmers Market,Italian Restaurant,Beer Bar,Restaurant,Café,Cheese Shop,Downtown Toronto,43.644771,-79.373306,0
1,"Brockton , Parkdale Village , Exhibition Place",Café,Breakfast Spot,Coffee Shop,Intersection,Stadium,Furniture / Home Store,Climbing Gym,Nightclub,Burrito Place,Restaurant,West Toronto,43.636847,-79.428191,0
2,Business reply mail Processing CentrE,Yoga Studio,Spa,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Park,Pizza Place,Restaurant,East Toronto,43.662744,-79.321558,0
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Airport,Boat or Ferry,Sculpture Garden,Rental Car Location,Plane,Airport Food Court,Coffee Shop,Downtown Toronto,43.628947,-79.394420,0
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Department Store,Thai Restaurant,Fried Chicken Joint,Bubble Tea Shop,Japanese Restaurant,Downtown Toronto,43.657952,-79.387383,0
5,Christie,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Gas Station,Candy Store,Nightclub,Baby Store,Restaurant,Downtown Toronto,43.669542,-79.422564,0
6,Church and Wellesley,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Restaurant,Yoga Studio,Hotel,Mediterranean Restaurant,Gay Bar,Gastropub,Men's Store,Downtown Toronto,43.665860,-79.383160,0
7,"Commerce Court , Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Deli / Bodega,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Downtown Toronto,43.648198,-79.379817,0
8,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Italian Restaurant,Sushi Restaurant,Café,Coffee Shop,Gym,Thai Restaurant,Gas Station,Central Toronto,43.704324,-79.388790,0
9,Davisville North,Breakfast Spot,Gym,Department Store,Food & Drink Shop,Hotel,Park,Sandwich Place,Men's Store,Mediterranean Restaurant,Metro Station,Central Toronto,43.712751,-79.390197,0


- Finally, let's visualize the resulting clusters:

In [153]:
# create map
map_clusters = folium.Map(location=[lat, lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(new_df['Latitude'], new_df['Longitude'], new_df['Neighborhood'], new_df['Cluster_label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## ==================== PART 3 COMPLETE ====================

- Just make sure:
    - to add enough Markdown cells to explain what you decided to do and to report any observations you make.
        - Added markdown cells for explanation.
    - to generate maps to visualize your neighborhoods and how they cluster together. 
        - Maps generated using Folium.